In [7]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from bs4 import BeautifulSoup
import os
import unicodedata
import string
import time
import lxml
import tqdm
import bs4
# Read recipe inputs
path = dataiku.Folder("7SAfnrwf").get_path()
path_of_csv = os.path.join(path, "base_html_06_04.csv")

df_html=pd.read_csv(path_of_csv)
np.unique(df_html.num_etude).shape

(148,)

In [8]:
#Extraction des strings beautifulsoup : liste des balises (et du texte qu'elles contiennent si il y'en a). Suppression des balises sans texte
# et de la liste des tailles de police (taille de la police i a l'indice i)


def txtBalise_extractor(file, limit = 3):
    soup = BeautifulSoup(file, features = "lxml")
    txt = soup.body.find_all(text = True,)

    fonts = soup.style.contents[0].split('\n')
    for id in range(len(fonts)):
        if 'font' in fonts[id]:
            first_font = id
            break
    fontsclean = fonts[first_font:len(fonts)-2]
    fontsizes = []
    for font in fontsclean:
        deb = font.index('font:')
        fin = font.index('pt')
        size = int(font[deb+5:fin])
        fontsizes.append(size)

    txt_clean = []
    features_doc = []
    longueur = len(txt)
    duration = 0.00001
    for line in txt:
        features = []
        if not '\n' in line:
            parent = line.parent
            parents = []
            if parent.get_text()==line:
                parents.append(parent)
                features += features_chain2(parent,fontsizes)

                k=0
                for k in range(limit):
                    parent = parent.parent
                    if parent.get_text().strip('\n')==line:
                        parents.append(parent)
                        features += features_chain2(parent,fontsizes)

                    else:
                        break
                txt_clean.append(parents[len(parents)-1])
                features.append(features_added(line))
                features_doc.append(features)
    return txt_clean,features_doc,fontsizes



def features_chain2(b4string,fontsizes):
    if type(b4string) != bs4.element.NavigableString:
        features = []
        features.append(b4string.name)
        attrs = b4string.attrs
        for attr in list(attrs.keys()):

            features.append(attr)
            attrsclean = feat_clean(attrs[attr],fontsizes)
            if type(attrsclean) == list:
                features += attrsclean
            elif type(attrsclean) != type(None):
                features.append(attrsclean)
        return(features)
    else:
        return

def feat_clean(feat,fontsizes):
    if type(feat) == list:
        if 'font' in str(feat):
            try :
                size = fontsizes[int(feat[0][4:])]
                return(size)
            except:
                pass
        return(feat[0])
    if 'bookmark' in str(feat):
        return('bookmark')

    if 'mailto' in str(feat):
        return

    if 'http' in str(feat):
        return

    if '.jpg' in str(feat):
        return

    if 'footnote' in str(feat):
        return

    if '.png' in str(feat):
        return

    if 'width:' in str(feat):
        return


    if type(feat) == str and len(feat.split(';')) >1:
        features = []
        for at in feat.split(';'):
            if at != '':
                features.append(at)
        return(features)

def features_added(stringtxt):
    # Feature construite à la main pour voir si il y'a un nombre dans la chaine
    for car in stringtxt:
            if car in '0123456789':
                return('hasNumber')

In [9]:
#Construit le dico des features uniques
def build_features(ftlists):
    diff_feat = []
    for entry in ftlists:
        for feats in entry:
            for feat in feats:
                diff_feat.append(str(feat))
    diff_feat = np.unique(diff_feat)

    k = 0
    feat_dico = {}
    for feat in diff_feat:
        feat_dico[feat] = k
        k += 1
    return(feat_dico)

In [10]:
#### Prend en entrée les features extraites ligne par ligne pour un doc,
#le dico des features, pour construire la matrice des indicatrices
def encode_matrix(doc_feat,features_names):
    for k in list(features_name.keys()):
        try:
            int(k)
        except:
            a=list(features_name.keys()).index(k)
            break
    matrix = np.zeros((len(doc_feat),len(list(features_names.keys())[a-1:])))
    k = 0
    for rowfeat in doc_feat:
        for feat in rowfeat:
            try:
                    feat=int(feat)
                    matrix[k,0] = feat
            except:
                    ind = features_names[str(feat)]-a
                    matrix[k,ind] = 1
        k += 1
    return(matrix)

#Applique la fonction précédente a une collection, concatène
def encode_collection(docs_feature_collection,feature_names):
    matrix = encode_matrix(docs_feature_collection[0],feature_names)
    i = 0
    for doc_feat in docs_feature_collection[1:]: #Pourquoi tu enlèves l'élément 0 ???
        child = encode_matrix(doc_feat,feature_names)
        matrix = np.concatenate((matrix,child),axis = 0)
    return(matrix)

In [11]:
#Extraction des textes dans les balises de liens "a"
def toc_extractor(file):
    soup = BeautifulSoup(file, features = "html")
    balises = soup.find_all('a',text = True)
    return [line.get_text().replace(u'\xa0',u'') for line in balises]

#Nettoie "a la main" les titres pour enlever les éléments qui n'en sont pas
def Title_clean(resultsTitle):
    resultsTitleClean = []
    for it,result in tqdm.tqdm(enumerate(resultsTitle),total = len(resultsTitle)) :
        if 'www' in result:
            pass
        else:
            try:
                int(result)
            except:
                for car in result:
                    if car.isnumeric()==True:
                        resultsTitleClean.append(result)
                        break
    return(resultsTitleClean)

In [12]:
#Encode 1 si c'est un titre, 0 sinon, pour 1 document
def Encode(results,resultsTitle):
    vector = []
    for res in results:
        clean = res.get_text()
        if clean in resultsTitle:
            vector.append(1)
        else:
            vector.append(0)
    return(vector)

#Applique la fonction précédente à la collection de documents, concatène
def Encode_vector(doc_collection,Title_collection):
    vector = []
    for doc,titles in zip(doc_collection,Title_collection):
        vector += Encode(doc,titles)
    return(vector)

In [13]:
start=time.time()
doc_collection = []
doc_styles = []
features_collection = []
for doc in tqdm.tqdm(df_html.texte):
    doc, features, fonts = txtBalise_extractor(doc)
    doc_collection.append(doc)
    features_collection.append(features)
    doc_styles.append(fonts)
end=time.time()
print('Durée :', round((end-start)/60,2),' minutes')

  9%|▉         | 14/148 [46:17<7:23:00, 198.36s/it]


KeyboardInterrupt: 

In [14]:
len(doc_collection),len(doc_collection[0]),len(doc_collection[0][0]),len(doc_styles),len(doc_styles[0])

(14, 6038, 1, 14, 38)

In [15]:
doc_collection

[[<title>EI Munchhouse_V3</title>,
  <h1><a name="bookmark0"></a><span class="font25" style="font-weight:bold;"><a name="bookmark1"></a>06</span></h1>,
  <p><span class="font20" style="font-weight:bold;">2019</span></p>,
  <p><span class="font14">Agence Aquitaine</span></p>,
  <p><span class="font14">SAFEGE</span></p>,
  <p><span class="font14">2A avenue de Berlincan</span></p>,
  <p><span class="font14">BP 50004</span></p>,
  <p><span class="font14">33166 SAINT MEDARD EN JALLES cedex</span></p>,
  <span class="font14">SAFEGE SAS - SIÈGE SOCIAL Parc dejület 15/27 rue du Port 92022 NANTERRE CEDEX </span>,
  <a href="http://www.safege.com"><span class="font14">www.safege.com</span></a>,
  <span class="font14"> ,.^l|</span>,
  <p><span class="font10" style="font-weight:bold;">CONSULTING</span></p>,
  <p><span class="font20">Projet d’implantation d’une centrale photovoltaïque au sol sur l’ancien aérodrome de Munchhouse (68)</span></p>,
  <p><span class="font20" style="font-weight:bold;">Et

In [16]:
import time 

start=time.time()

features_name = build_features(features_collection)

end=time.time()
print('Durée :', round((end-start)/60,2),' minutes')

Durée : 0.04  minutes


In [25]:
import pickle
path2 = dataiku.Folder("lZ0B3sSL").get_path()
pickle.dump(features_name,open(path2+'/features_name.pickle','wb'))

In [26]:
features_name

{'10': 0,
 '11': 1,
 '112': 2,
 '115': 3,
 '12': 4,
 '127': 5,
 '13': 6,
 '14': 7,
 '15': 8,
 '16': 9,
 '17': 10,
 '18': 11,
 '19': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '30': 23,
 '31': 24,
 '32': 25,
 '33': 26,
 '34': 27,
 '35': 28,
 '36': 29,
 '37': 30,
 '38': 31,
 '39': 32,
 '4': 33,
 '40': 34,
 '42': 35,
 '43': 36,
 '44': 37,
 '45': 38,
 '47': 39,
 '48': 40,
 '5': 41,
 '51': 42,
 '56': 43,
 '6': 44,
 '61': 45,
 '67': 46,
 '68': 47,
 '7': 48,
 '72': 49,
 '74': 50,
 '8': 51,
 '9': 52,
 'None': 53,
 'a': 54,
 'bookmark': 55,
 'border-bottom:solid': 56,
 'border-right:solid': 57,
 'border-top:solid': 58,
 'border:solid': 59,
 'class': 60,
 'colspan': 61,
 'div': 62,
 'font-style:italic': 63,
 'font-variant:small-caps': 64,
 'font-weight:bold': 65,
 'h1': 66,
 'h2': 67,
 'h3': 68,
 'h4': 69,
 'h5': 70,
 'h6': 71,
 'hasNumber': 72,
 'href': 73,
 'li': 74,
 'list-style:none': 75,
 'p': 76,
 'rowspan': 77,
 'span

In [185]:
start=time.time()

Title_collection = []

for doc in df_html.texte:
    Title_collection.append(Title_clean(toc_extractor(doc)))

end=time.time()
print('Durée :', round((end-start)/60,2),' minutes')

100%|██████████| 90/90 [00:00<00:00, 124214.33it/s]
0it [00:00, ?it/s]
100%|██████████| 246/246 [00:00<00:00, 117183.28it/s]
0it [00:00, ?it/s]
100%|██████████| 413/413 [00:00<00:00, 124900.68it/s]

Durée : 7.68  minutes


In [0]:
features_collection

In [186]:
start=time.time()

ft_matrix = encode_collection(features_collection,features_name)

target_vector = Encode_vector(doc_collection,Title_collection)


end=time.time()
print('Durée :', round((end-start)/60,2),' minutes')

Durée : 2.15  minutes


In [187]:
df_html.texte

0      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
1      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
2      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
3      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
4      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
5      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
6      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
7      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
8      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
9      ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
10     ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
11     ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
12     ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
13     ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
14     ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
15     ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
16     ﻿<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 T...
17     ﻿<!DOCTYPE HTML PUBLIC "

In [191]:
phrases = []
doc_id = []
for ids,doc in zip(df_html.num_etude,doc_collection):
    for phrase in doc:
        phrases.append(str(phrase))
        doc_id.append(ids)

df_phrase_id = pd.DataFrame({'phrase':phrases, 'num_etude':doc_id})
df_phrase_id

,phrase,num_etude
0,<title>EI Munchhouse_V3</title>,1003691
1,"<h1><a name=""bookmark0""></a><span class=""font2...",1003691
2,"<p><span class=""font20"" style=""font-weight:bol...",1003691
3,"<p><span class=""font14"">Agence Aquitaine</span...",1003691
4,"<p><span class=""font14"">SAFEGE</span></p>",1003691
...,...,...
2443029,"<span class=""font21"">Disponible sur le site de...",238301
2443030,"<a href=""http://www.clefdeschamps.info/""><span...",238301
2443031,"<p><sup><a href=""#footnote14"">14</a></sup><a n...",238301
2443032,"<span class=""font0"" style=""font-weight:bold;"">...",238301


In [198]:
df=pd.DataFrame(ft_matrix)
df=pd.concat([df_phrase_id,df],axis=1)
len(np.unique(df.num_etude))

148

In [199]:
df

,phrase,num_etude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,<title>EI Munchhouse_V3</title>,1003691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"<h1><a name=""bookmark0""></a><span class=""font2...",1003691,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"<p><span class=""font20"" style=""font-weight:bol...",1003691,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"<p><span class=""font14"">Agence Aquitaine</span...",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"<p><span class=""font14"">SAFEGE</span></p>",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443029,"<span class=""font21"">Disponible sur le site de...",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2443030,"<a href=""http://www.clefdeschamps.info/""><span...",238301,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2443031,"<p><sup><a href=""#footnote14"">14</a></sup><a n...",238301,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2443032,"<span class=""font0"" style=""font-weight:bold;"">...",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [200]:
base_phrase_clean_df = df # Compute a Pandas dataframe to write into base_phrase_clean


# Write recipe outputs
base_phrase_clean = dataiku.Dataset("base_phrase_clean")
base_phrase_clean.write_with_schema(base_phrase_clean_df)

2443034 rows successfully written (LhgNnSuTBT)
